<a href="https://colab.research.google.com/github/georgejordan3/IBM_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighboorhoods in Toronto

George Jordan <br>
IBM Applied Data Science Capstone <br>
Last Updated: 2-21-21 <br>

This notebook will demonstrate the methods I used to cluster the neighborhoods of Toronto. First, I used pandas to scrape a wikipedia article for data to be implemented into a localized dataframe. Then, I added geocoordinates from a .csv file to be added to the dataframe.

## Scraping and Creating the Dataframe, Part 1

First, I will import the libraries necessary for scraping the URL.

In [1]:
import pandas as pd

Using pandas, I imported the table into a dataframe and displayed the first five rows to ensure that it was made properly.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


I removed any row that had an unassigned Borough. After that, I checked the size of the dataframe before and after the removal of rows.

In [3]:
df1 = df[df.Borough != "Not assigned"]
print(df.shape)
print(df1.shape)

(180, 3)
(103, 3)


Next, we will combine Boroughs that share a zip code so that we only have one row per zipcode.

In [4]:
combined = df1.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
combined.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


For every row that has a named Borough but an unnamed Neighbourhood, I made the name of the Neighbourhood correspond with the Borough.

In [5]:
empty = combined['Neighbourhood'] == "Not assigned"
combined.loc[empty, 'Neighbourhood'] = combined.loc[empty, 'Borough']

In [6]:
combined.shape

(103, 3)

This concludes the first part of this assignment.

## Adding Coordinates to the Data, Part 2

Using the provided link to .csv data for the coordinates of the postal codes, I made a dataframe from this data.

In [7]:
url = "https://cocl.us/Geospatial_data"
coord = pd.read_csv(url)
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I then combined the two dataframes. Using an inner join, I ensured that only the rows represented in both dataframes would be present in the combined dataframe.

In [8]:
postal_coord = pd.merge(combined, coord)
postal_coord.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


To ensure that I didn't lose or gain any rows, I once again checked the shape of the dataframe.

In [9]:
postal_coord.shape

(103, 5)

This concludes the second part of the assignment.

## Clustering the Neighbourhoods in Toronto

For this section, I had to import the geopy and folium libraries in order to source some coordinates as well as display the clustering over a map of the area.

In [19]:
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster

Using geopy, I found the location of Toronto and then stored the coordinates in variables to be used in the ongoing exploration. I then used the folium library to display a map to be populated later.

In [20]:
geolocator = Nominatim(user_agent="toronto_clustering")
location = geolocator.geocode("Toronto")
latitude = location.latitude
longitude = location.longitude
print(location.address)
print((location.latitude, location.longitude))

Toronto, Golden Horseshoe, Ontario, Canada
(43.6534817, -79.3839347)


In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In order to get some labels on the map, I created a list of coordinates so that I could run a for-loop iterating through the list to place the points on the map. I rechecked the length to make sure nothing was lost.

In [22]:
neighbourhoods = postal_coord[['Latitude', 'Longitude']]
locationlist = neighbourhoods.values.tolist()
len(locationlist)

103

In [23]:
for point in range(0, len(locationlist)):
  folium.Marker(locationlist[point], popup = postal_coord['Neighbourhood'][point]).add_to(map_toronto)
map_toronto

Since our directions for this project were pretty open ended and merely asked for us to cluster the data as we liked, I opted not to use an API call for Foursquare data into determining venues, similarly to the lab using Manhattan data. Instead, I used a folium plugin that automatically clusters the data. Zoom in and out to see how the dynamic clusters change their display.

In [26]:
from folium.plugins import MarkerCluster
map_toronto2 = folium.Map(location=[latitude, longitude], tiles='CartoDB dark_matter', zoom_start=10)
marker_cluster = folium.plugins.MarkerCluster().add_to(map_toronto2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], postal_coord['Neighbourhood'][point]).add_to(marker_cluster)
map_toronto2